In [ ]:
# Cell 1: Environment Setup
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import psutil
import gc
import warnings
warnings.filterwarnings('ignore')

def create_project_dirs():
    """Create project directories if they don't exist"""
    base_dirs = ['data', 'plots', 'animations', 'models', 'results']
    for d in base_dirs:
        os.makedirs(d, exist_ok=True)

def get_memory_usage():
    """Return current memory usage in MB"""
    return psutil.Process(os.getpid()).memory_info().rss / 1024 / 1024

def clear_memory():
    """Clear unused memory"""
    gc.collect()
    
def set_plotting_style():
    """Set consistent plotting style"""
    plt.style.use('ggplot')
    plt.rcParams['figure.figsize'] = [10, 6]
    plt.rcParams['figure.dpi'] = 100
    plt.rcParams['font.size'] = 10
    
# NFC West teams
NFC_WEST = ['ARI', 'LAR', 'SF', 'SEA']

# Initialize environment
print(f"Initial memory usage: {get_memory_usage():.2f} MB")
create_project_dirs()
set_plotting_style()
print("\nProject directories created:")
print("\n".join(os.listdir()))
print(f"\nFinal memory usage: {get_memory_usage():.2f} MB")

# Verify pandas version and key settings
print("\nEnvironment Info:")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")
pd.set_option('display.max_columns', None)


In [ ]:
# Cell 2: Data Loading & Filtering
import os

# Print current working directory for verification
print("Current working directory:", os.getcwd())

# Update NFC West teams constant - using 'LA' instead of 'LAR'
NFC_WEST = ['ARI', 'LA', 'SF', 'SEA']

# Define data directory using relative path (up one level)
DATA_DIR = os.path.join(os.path.dirname(os.getcwd()), 'data')
print(f"\nUsing data directory: {DATA_DIR}")


def load_nfc_west_games(weeks_range=[1, 2, 3, 4, 5]):
    """Load and filter games for NFC West teams"""
    print("Loading games data...")
    games_path = os.path.join(DATA_DIR, 'games.csv')
    games_df = pd.read_csv(games_path)
    
    # Filter for NFC West games
    nfc_west_mask = (games_df['homeTeamAbbr'].isin(NFC_WEST) | 
                     games_df['visitorTeamAbbr'].isin(NFC_WEST))
    weeks_mask = games_df['week'].isin(weeks_range)
    
    filtered_games = games_df[nfc_west_mask & weeks_mask].copy()
    
    print(f"\nFound {len(filtered_games)} NFC West games in weeks {weeks_range}")
    return filtered_games[['gameId', 'week', 'homeTeamAbbr', 'visitorTeamAbbr']]

def load_tracking_data(games_df, week):
    """Load tracking data for specific week with memory optimization"""
    print(f"\nProcessing week {week}")
    print(f"Memory before loading: {get_memory_usage():.2f} MB")
    
    tracking_file = os.path.join(DATA_DIR, f'tracking_week_{week}.csv')
    relevant_game_ids = set(games_df['gameId'])
    
    # Read with chunking for memory efficiency
    chunks = []
    chunk_size = 100000
    
    for chunk in tqdm(pd.read_csv(tracking_file, chunksize=chunk_size)):
        # Filter for relevant games
        filtered_chunk = chunk[chunk['gameId'].isin(relevant_game_ids)].copy()
        chunks.append(filtered_chunk)
        
        # Clear memory
        del filtered_chunk
        clear_memory()
        
        # Check memory usage
        if get_memory_usage() > 32000:  # 32GB warning
            print("Warning: High memory usage detected")
    
    # Combine chunks
    week_data = pd.concat(chunks, ignore_index=True)
    clear_memory()
    
    print(f"Memory after loading: {get_memory_usage():.2f} MB")
    return week_data

def save_processed_data(df, filename):
    """Save processed data to parquet format"""
    output_path = os.path.join(DATA_DIR, filename)
    df.to_parquet(output_path, index=False)
    print(f"Saved to {output_path}")

# Execute data loading
print(f"Starting memory usage: {get_memory_usage():.2f} MB")

# Load game data
games = load_nfc_west_games()
save_processed_data(games, 'nfc_west_games.parquet')

# Process each week
all_tracking_data = []
for week in range(1, 6):
    week_data = load_tracking_data(games, week)
    
    # Save week data
    save_processed_data(week_data, f'nfc_west_week_{week}.parquet')
    
    # Keep summary stats
    summary = {
        'week': week,
        'n_plays': week_data['playId'].nunique(),
        'n_players': week_data['nflId'].nunique(),
        'n_frames': len(week_data)
    }
    all_tracking_data.append(summary)
    
    # Clear memory
    del week_data
    clear_memory()

# Create summary DataFrame
summary_df = pd.DataFrame(all_tracking_data)
print("\nData Loading Summary:")
print(summary_df)

# Save summary to results directory (up one level, then into results)
RESULTS_DIR = os.path.join(os.path.dirname(os.getcwd()), 'results')
os.makedirs(RESULTS_DIR, exist_ok=True)
summary_df.to_csv(os.path.join(RESULTS_DIR, 'data_loading_summary.csv'), index=False)
print(f"\nFinal memory usage: {get_memory_usage():.2f} MB")

# Display NFC West game distribution
team_games = pd.concat([
    games['homeTeamAbbr'].value_counts(),
    games['visitorTeamAbbr'].value_counts()
], axis=1)
team_games.columns = ['Home Games', 'Away Games']
team_games.fillna(0, inplace=True)
team_games['Total Games'] = team_games.sum(axis=1)
team_games = team_games.loc[NFC_WEST]

print("\nNFC West Game Distribution:")
print(team_games)
team_games.to_csv(os.path.join(RESULTS_DIR, 'nfc_west_game_distribution.csv'))


In [ ]:
# Cell 3: Basic Movement Analysis
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

def load_players_data():
    """Load and filter for defensive players"""
    players_path = os.path.join(DATA_DIR, 'players.csv')
    players_df = pd.read_csv(players_path)
    
    # Define defensive positions
    defensive_positions = ['SS', 'FS', 'CB', 'DB', 'S', 'ILB', 'LB', 'MLB', 'OLB']
    return players_df[players_df['position'].isin(defensive_positions)]

def calculate_movement_metrics(week_data, defensive_players):
    """Calculate basic movement metrics for defensive players"""
    # Filter for defensive players
    defensive_data = week_data[week_data['nflId'].isin(defensive_players['nflId'])]
    
    # Calculate metrics by player
    metrics = []
    for player_id, player_data in defensive_data.groupby('nflId'):
        player_metrics = {
            'nflId': player_id,
            'position': defensive_players[defensive_players['nflId'] == player_id]['position'].iloc[0],
            'team': player_data['club'].iloc[0],
            'avg_speed': player_data['s'].mean(),
            'max_speed': player_data['s'].max(),
            'avg_acceleration': player_data['a'].mean(),
            'direction_changes': np.sum(np.abs(np.diff(player_data['dir'])) > 10)
        }
        metrics.append(player_metrics)
    
    return pd.DataFrame(metrics)

# Load defensive players
defensive_players = load_players_data()
print(f"Found {len(defensive_players)} defensive players")

# Process each week's data
all_metrics = []
for week in range(1, 6):
    print(f"\nProcessing week {week}")
    
    # Load week data from parquet
    week_data_path = os.path.join(DATA_DIR, f'nfc_west_week_{week}.parquet')
    week_data = pd.read_parquet(week_data_path)
    
    # Calculate metrics
    week_metrics = calculate_movement_metrics(week_data, defensive_players)
    week_metrics['week'] = week
    all_metrics.append(week_metrics)
    
    # Clear memory
    del week_data
    clear_memory()

# Combine all weeks
movement_df = pd.concat(all_metrics, ignore_index=True)

# Create speed distribution plot (Plot 1)
plt.figure(figsize=(12, 8))
sns.boxplot(data=movement_df, x='team', y='avg_speed', hue='position')
plt.title('Defensive Speed Distribution by Team and Position')
plt.xlabel('Team')
plt.ylabel('Average Speed (yards/second)')
plt.xticks(rotation=45)
plt.legend(title='Position', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.savefig(os.path.join(os.path.dirname(os.getcwd()), 'plots', 'defensive_speed_distribution.png'))
plt.close()

# Save metrics to results
metrics_path = os.path.join(os.path.dirname(os.getcwd()), 'results', 'movement_metrics.csv')
movement_df.to_csv(metrics_path, index=False)

# Print summary statistics
print("\nMovement Metrics Summary by Team:")
team_summary = movement_df.groupby('team').agg({
    'avg_speed': ['mean', 'std'],
    'max_speed': ['mean', 'std'],
    'direction_changes': ['mean', 'std']
}).round(2)
print(team_summary)

# Print memory usage
print(f"\nFinal memory usage: {get_memory_usage():.2f} MB")


In [ ]:
# Cell 4: Zone Coverage Analysis (Optimized)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull
import seaborn as sns
from tqdm import tqdm

def create_coverage_heatmap(tracking_df, team, save_path):
    """Create defensive coverage heatmap for a team using sampled data"""
    # Sample data points for faster processing (e.g., every 10th frame)
    sampled_data = tracking_df[tracking_df['club'] == team].iloc[::10]
    
    plt.figure(figsize=(15, 8))
    
    # Plot football field boundaries
    plt.plot([0, 0], [0, 53.3], 'white', alpha=0.5)
    plt.plot([120, 120], [0, 53.3], 'white', alpha=0.5)
    plt.plot([0, 120], [0, 0], 'white', alpha=0.5)
    plt.plot([0, 120], [53.3, 53.3], 'white', alpha=0.5)
    
    # Create heatmap with fewer bins
    plt.hist2d(sampled_data['x'], sampled_data['y'], 
               bins=40, range=[[0, 120], [0, 53.3]], 
               cmap='YlOrRd', alpha=0.7)
    
    plt.colorbar(label='Defensive Position Frequency')
    plt.title(f'{team} Defensive Coverage Heatmap')
    plt.xlabel('Field Position (yards)')
    plt.ylabel('Field Position (yards)')
    
    ax = plt.gca()
    ax.set_facecolor('forestgreen')
    
    plt.savefig(save_path, bbox_inches='tight', dpi=300)
    plt.close()

def calculate_coverage_metrics(tracking_df, team):
    """Calculate coverage metrics using sampled plays"""
    team_data = tracking_df[tracking_df['club'] == team]
    
    # Sample plays for analysis (e.g., every 5th play)
    play_ids = team_data['playId'].unique()[::5]
    
    metrics = {
        'team': team,
        'total_plays': len(play_ids),
        'avg_defenders': team_data.groupby('playId')['nflId'].nunique().mean(),
        'coverage_area': 0,
        'avg_spacing': 0
    }
    
    # Calculate metrics for sampled plays
    coverage_areas = []
    spacings = []
    
    for play_id in play_ids:
        play_data = team_data[team_data['playId'] == play_id]
        if len(play_data) > 0:
            # Sample frames for this play
            sampled_frames = play_data.iloc[::5]
            points = sampled_frames[['x', 'y']].values
            
            if len(points) >= 3:
                try:
                    hull = ConvexHull(points)
                    coverage_areas.append(hull.area)
                    
                    # Calculate spacing for subset of points
                    if len(points) > 10:
                        points = points[:10]
                    spacing = np.mean([np.min([np.linalg.norm(p1 - p2) 
                                             for p2 in points if not np.array_equal(p1, p2)]) 
                                    for p1 in points])
                    spacings.append(spacing)
                except:
                    continue
    
    metrics['coverage_area'] = np.mean(coverage_areas) if coverage_areas else 0
    metrics['avg_spacing'] = np.mean(spacings) if spacings else 0
    
    return metrics

# Process all weeks data first
print("Loading tracking data...")
all_tracking_data = []
for week in tqdm(range(1, 6), desc="Loading weeks"):
    week_data_path = os.path.join(DATA_DIR, f'nfc_west_week_{week}.parquet')
    week_data = pd.read_parquet(week_data_path)
    all_tracking_data.append(week_data)
    clear_memory()

# Combine all weeks
tracking_df = pd.concat(all_tracking_data, ignore_index=True)
del all_tracking_data
clear_memory()

# Process each team
team_metrics = []
for team in tqdm(NFC_WEST, desc="Processing teams"):
    print(f"\nAnalyzing {team}")
    
    # Create heatmap
    heatmap_path = os.path.join(os.path.dirname(os.getcwd()), 'plots', f'{team}_coverage_heatmap.png')
    create_coverage_heatmap(tracking_df, team, heatmap_path)
    
    # Calculate metrics
    metrics = calculate_coverage_metrics(tracking_df, team)
    team_metrics.append(metrics)
    clear_memory()

# Create metrics DataFrame
metrics_df = pd.DataFrame(team_metrics)

# Create summary visualization
plt.figure(figsize=(12, 6))
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Coverage area comparison
sns.barplot(data=metrics_df, x='team', y='coverage_area', ax=ax1)
ax1.set_title('Average Coverage Area by Team')
ax1.set_ylabel('Coverage Area (sq. yards)')

# Defender spacing comparison
sns.barplot(data=metrics_df, x='team', y='avg_spacing', ax=ax2)
ax2.set_title('Average Defender Spacing by Team')
ax2.set_ylabel('Spacing (yards)')

plt.tight_layout()
plt.savefig(os.path.join(os.path.dirname(os.getcwd()), 'plots', 'coverage_metrics_comparison.png'))
plt.close()

# Save metrics to results
metrics_df.to_csv(os.path.join(os.path.dirname(os.getcwd()), 'results', 'coverage_metrics.csv'), index=False)

# Print summary and identify top teams
print("\nCoverage Metrics Summary:")
print(metrics_df.round(2))

# Rank teams based on combined metrics
metrics_df['overall_score'] = (
    metrics_df['coverage_area'] / metrics_df['coverage_area'].max() +
    metrics_df['avg_spacing'] / metrics_df['avg_spacing'].max()
) / 2

top_teams = metrics_df.nlargest(2, 'overall_score')
print("\nTop 2 performing teams:")
print(top_teams[['team', 'overall_score']].round(3))

print(f"\nFinal memory usage: {get_memory_usage():.2f} MB")


In [ ]:
# Cell 5: Defender Tracking - Focus on Top Team
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# Get top performing team from previous analysis
top_team = metrics_df.nlargest(1, 'overall_score')['team'].iloc[0]
print(f"Focusing analysis on top team: {top_team}")

def find_notable_plays(tracking_df, plays_df, team):
    """Find notable defensive plays based on movement patterns"""
    # Merge tracking with plays data
    defensive_plays = plays_df[plays_df['defensiveTeam'] == team][['gameId', 'playId', 'passResult']]
    play_data = pd.merge(tracking_df[tracking_df['club'] == team], 
                        defensive_plays, on=['gameId', 'playId'])
    
    play_metrics = []
    
    # Analyze each play
    for (game_id, play_id), play_group in tqdm(play_data.groupby(['gameId', 'playId']), 
                                              desc="Analyzing plays"):
        # Calculate movement metrics
        avg_speed = play_group['s'].mean()
        max_speed = play_group['s'].max()
        direction_changes = np.sum(np.abs(np.diff(play_group['dir'])) > 10)
        
        # Calculate defender spread
        frames = play_group.groupby('frameId')
        avg_spread = frames.apply(lambda x: np.std(x[['x', 'y']].values)).mean()
        
        play_metrics.append({
            'gameId': game_id,
            'playId': play_id,
            'avg_speed': avg_speed,
            'max_speed': max_speed,
            'direction_changes': direction_changes,
            'defender_spread': avg_spread,
            'pass_result': play_group['passResult'].iloc[0],
            'n_frames': len(frames)
        })
    
    return pd.DataFrame(play_metrics)

def plot_notable_play(tracking_df, play_info, save_path):
    """Create trajectory visualization for a notable play"""
    play_data = tracking_df[(tracking_df['gameId'] == play_info['gameId']) & 
                           (tracking_df['playId'] == play_info['playId'])]
    
    plt.figure(figsize=(15, 8))
    
    # Plot field
    plt.plot([0, 0], [0, 53.3], 'white', alpha=0.5)
    plt.plot([120, 120], [0, 53.3], 'white', alpha=0.5)
    plt.plot([0, 120], [0, 0], 'white', alpha=0.5)
    plt.plot([0, 120], [53.3, 53.3], 'white', alpha=0.5)
    
    # Plot defender trajectories
    for player_id, player_data in play_data.groupby('nflId'):
        positions = player_data[['x', 'y']].values
        plt.plot(positions[:, 0], positions[:, 1], '-', alpha=0.6, linewidth=2)
        
        # Mark start and end positions
        plt.scatter(positions[0, 0], positions[0, 1], c='green', s=100, label='Start' if player_id == play_data['nflId'].iloc[0] else "")
        plt.scatter(positions[-1, 0], positions[-1, 1], c='red', s=100, label='End' if player_id == play_data['nflId'].iloc[0] else "")
    
    plt.title(f"Defensive Trajectories - {top_team}\nPlay ID: {play_info['playId']} (Pass Result: {play_info['pass_result']})")
    plt.xlabel('Field Position (yards)')
    plt.ylabel('Field Position (yards)')
    
    ax = plt.gca()
    ax.set_facecolor('forestgreen')
    plt.legend()
    
    plt.savefig(save_path, bbox_inches='tight', dpi=300)
    plt.close()

# Load necessary data
print("Loading data...")
tracking_df = pd.read_parquet(os.path.join(DATA_DIR, 'nfc_west_week_1.parquet'))  # Start with week 1
plays_df = pd.read_csv(os.path.join(DATA_DIR, 'plays.csv'))

# Find notable plays
play_metrics = find_notable_plays(tracking_df, plays_df, top_team)

# Select top 3 plays based on different criteria
interesting_plays = pd.concat([
    play_metrics.nlargest(1, 'defender_spread'),  # Most spread out defense
    play_metrics.nlargest(1, 'direction_changes'),  # Most complex movements
    play_metrics[play_metrics['pass_result'].isin(['I', 'IN'])].nlargest(1, 'max_speed')  # Successful defense with high speed
])

# Create visualizations for selected plays
print("\nCreating play visualizations...")
for idx, play in interesting_plays.iterrows():
    save_path = os.path.join(os.path.dirname(os.getcwd()), 'plots', 
                            f"{top_team}_play_{play['playId']}_trajectories.png")
    plot_notable_play(tracking_df, play, save_path)

# Save play metrics
interesting_plays.to_csv(os.path.join(os.path.dirname(os.getcwd()), 'results', 
                                    f'{top_team}_notable_plays.csv'), index=False)

print("\nPlay Analysis Summary:")
print(interesting_plays[['playId', 'pass_result', 'defender_spread', 
                        'direction_changes', 'max_speed']].round(2))

print(f"\nFinal memory usage: {get_memory_usage():.2f} MB")


In [ ]:
# Cell 6: Formation Analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial import ConvexHull
from tqdm import tqdm

def analyze_formation(play_tracking, frame_id):
    """Analyze defensive formation for a specific frame"""
    frame_data = play_tracking[play_tracking['frameId'] == frame_id]
    
    if len(frame_data) < 3:  # Need at least 3 defenders for meaningful formation
        return None
        
    positions = frame_data[['x', 'y']].values
    
    # Calculate formation metrics
    try:
        hull = ConvexHull(positions)
        formation_metrics = {
            'area': hull.area,  # Area covered by formation
            'width': np.ptp(positions[:, 0]),  # Formation width
            'depth': np.ptp(positions[:, 1]),  # Formation depth
            'avg_spacing': np.mean([np.min([np.linalg.norm(p1 - p2) 
                                          for j, p2 in enumerate(positions) if i != j])
                                  for i, p1 in enumerate(positions)]),
            'n_defenders': len(positions)
        }
        return formation_metrics
    except:
        return None

def plot_formation_analysis(play_tracking, play_info, save_path):
    """Create formation analysis visualization"""
    # Get pre-snap formation
    pre_snap = play_tracking[play_tracking['frameType'] == 'BEFORE_SNAP']
    if len(pre_snap) == 0:
        return
        
    # Get last frame before snap
    last_pre_snap = pre_snap.groupby('frameId').last().reset_index()
    
    plt.figure(figsize=(15, 8))
    
    # Plot field
    plt.plot([0, 0], [0, 53.3], 'white', alpha=0.5)
    plt.plot([120, 120], [0, 53.3], 'white', alpha=0.5)
    plt.plot([0, 120], [0, 0], 'white', alpha=0.5)
    plt.plot([0, 120], [53.3, 53.3], 'white', alpha=0.5)
    
    # Plot defender positions
    positions = last_pre_snap[['x', 'y']].values
    plt.scatter(positions[:, 0], positions[:, 1], c='white', s=100, label='Defenders')
    
    # Plot formation hull
    if len(positions) >= 3:
        hull = ConvexHull(positions)
        for simplex in hull.simplices:
            plt.plot(positions[simplex, 0], positions[simplex, 1], 'w--', alpha=0.5)
    
    # Add formation metrics to plot
    metrics = analyze_formation(play_tracking, last_pre_snap['frameId'].iloc[0])
    if metrics:
        plt.title(f"Defensive Formation Analysis - {top_team}\n" +
                 f"Width: {metrics['width']:.1f} yards, Depth: {metrics['depth']:.1f} yards\n" +
                 f"Area: {metrics['area']:.1f} sq. yards, Avg Spacing: {metrics['avg_spacing']:.1f} yards")
    
    plt.xlabel('Field Position (yards)')
    plt.ylabel('Field Position (yards)')
    
    ax = plt.gca()
    ax.set_facecolor('forestgreen')
    
    plt.savefig(save_path, bbox_inches='tight', dpi=300)
    plt.close()

# Load data for the top team's plays
print(f"Analyzing formations for {top_team}...")

# Use the notable plays identified in Cell 5
notable_plays_df = pd.read_csv(os.path.join(os.path.dirname(os.getcwd()), 
                                           'results', f'{top_team}_notable_plays.csv'))

# Analyze formations for each notable play
formation_metrics = []
for _, play in tqdm(notable_plays_df.iterrows(), desc="Analyzing formations"):
    # Load play tracking data
    play_tracking = tracking_df[
        (tracking_df['gameId'] == play['gameId']) & 
        (tracking_df['playId'] == play['playId']) &
        (tracking_df['club'] == top_team)
    ]
    
    # Create formation visualization
    save_path = os.path.join(os.path.dirname(os.getcwd()), 'plots', 
                            f"{top_team}_play_{play['playId']}_formation.png")
    plot_formation_analysis(play_tracking, play, save_path)
    
    # Analyze pre-snap formation
    pre_snap_frames = play_tracking[play_tracking['frameType'] == 'BEFORE_SNAP']['frameId'].unique()
    if len(pre_snap_frames) > 0:
        metrics = analyze_formation(play_tracking, pre_snap_frames[-1])  # Last pre-snap frame
        if metrics:
            metrics['gameId'] = play['gameId']
            metrics['playId'] = play['playId']
            metrics['pass_result'] = play['pass_result']
            formation_metrics.append(metrics)

# Create formation metrics DataFrame
formation_df = pd.DataFrame(formation_metrics)

# Create summary visualization
plt.figure(figsize=(12, 8))
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# Formation area vs success
sns.boxplot(data=formation_df, x='pass_result', y='area', ax=ax1)
ax1.set_title('Formation Area by Play Result')
ax1.set_xlabel('Pass Result')
ax1.set_ylabel('Area (sq. yards)')

# Formation spacing distribution
sns.histplot(data=formation_df, x='avg_spacing', bins=20, ax=ax2)
ax2.set_title('Distribution of Defender Spacing')
ax2.set_xlabel('Average Spacing (yards)')

# Width vs Depth scatter
sns.scatterplot(data=formation_df, x='width', y='depth', 
                size='area', sizes=(100, 1000), alpha=0.6, ax=ax3)
ax3.set_title('Formation Dimensions')
ax3.set_xlabel('Width (yards)')
ax3.set_ylabel('Depth (yards)')

# Number of defenders
sns.countplot(data=formation_df, x='n_defenders', ax=ax4)
ax4.set_title('Number of Defenders in Formation')
ax4.set_xlabel('Number of Defenders')

plt.tight_layout()
plt.savefig(os.path.join(os.path.dirname(os.getcwd()), 'plots', 
                        f'{top_team}_formation_analysis.png'))
plt.close()

# Save formation metrics
formation_df.to_csv(os.path.join(os.path.dirname(os.getcwd()), 'results', 
                                f'{top_team}_formation_metrics.csv'), index=False)

print("\nFormation Analysis Summary:")
print(formation_df.describe().round(2))
print(f"\nFinal memory usage: {get_memory_usage():.2f} MB")


In [ ]:
# Cell 7: Environmental Constraints Analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import Voronoi
import matplotlib.patches as patches
from tqdm import tqdm

def calculate_safe_zones(positions, field_dims=(120, 53.3)):
    """Calculate areas where defenders can move safely"""
    try:
        # Create Voronoi diagram for defensive positions
        vor = Voronoi(positions)
        
        # Initialize bounds
        bounds = [0, field_dims[0], 0, field_dims[1]]
        
        # Filter vertices within field bounds
        valid_vertices = []
        for vertex in vor.vertices:
            if (bounds[0] <= vertex[0] <= bounds[1] and 
                bounds[2] <= vertex[3] <= bounds[3]):
                valid_vertices.append(vertex)
                
        return np.array(valid_vertices) if valid_vertices else None
    except:
        return None

def analyze_constraints(play_tracking, field_dims=(120, 53.3)):
    """Analyze environmental constraints on defender movement"""
    constraints = {
        'sideline_proximity': [],  # Distance to nearest sideline
        'defender_spacing': [],    # Distance to nearest defender
        'movement_angles': [],     # Available movement angles
        'field_position': []      # Position relative to field length
    }
    
    for _, frame in play_tracking.groupby('frameId'):
        positions = frame[['x', 'y']].values
        
        for pos in positions:
            # Sideline proximity
            sideline_dist = min(pos[1], field_dims[1] - pos[1])
            constraints['sideline_proximity'].append(sideline_dist)
            
            # Defender spacing
            other_positions = positions[~np.all(positions == pos, axis=1)]
            if len(other_positions) > 0:
                min_spacing = np.min(np.linalg.norm(other_positions - pos, axis=1))
                constraints['defender_spacing'].append(min_spacing)
            
            # Field position
            field_pos = pos[0] / field_dims[0]
            constraints['field_position'].append(field_pos)
            
            # Movement angles (simplified)
            available_angles = []
            for angle in range(0, 360, 45):
                rad = np.radians(angle)
                direction = np.array([np.cos(rad), np.sin(rad)])
                available_angles.append(angle)
            constraints['movement_angles'].append(len(available_angles))
    
    return constraints

def plot_constraints(play_tracking, constraints, save_path):
    """Visualize defensive movement constraints"""
    plt.figure(figsize=(15, 10))
    
    # Create subplot grid
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
    
    # 1. Sideline Proximity Distribution
    sns.histplot(constraints['sideline_proximity'], bins=20, ax=ax1)
    ax1.set_title('Distance to Sideline Distribution')
    ax1.set_xlabel('Distance to Sideline (yards)')
    
    # 2. Defender Spacing Distribution
    sns.histplot(constraints['defender_spacing'], bins=20, ax=ax2)
    ax2.set_title('Inter-Defender Spacing Distribution')
    ax2.set_xlabel('Distance to Nearest Defender (yards)')
    
    # 3. Field Position Heatmap
    positions = play_tracking[['x', 'y']].values
    ax3.hist2d(positions[:, 0], positions[:, 1], bins=30, cmap='YlOrRd')
    ax3.set_title('Defensive Position Heatmap')
    ax3.set_xlabel('Field Position (yards)')
    ax3.set_ylabel('Field Width (yards)')
    
    # 4. Movement Constraints Visualization
    sample_frame = play_tracking.groupby('frameId').first().iloc[0]
    frame_data = play_tracking[play_tracking['frameId'] == sample_frame.name]
    positions = frame_data[['x', 'y']].values
    
    ax4.scatter(positions[:, 0], positions[:, 1], c='white', s=100)
    
    # Add field boundaries
    for ax in [ax3, ax4]:
        ax.plot([0, 0], [0, 53.3], 'white', alpha=0.5)
        ax.plot([120, 120], [0, 53.3], 'white', alpha=0.5)
        ax.plot([0, 120], [0, 0], 'white', alpha=0.5)
        ax.plot([0, 120], [53.3, 53.3], 'white', alpha=0.5)
        ax.set_facecolor('forestgreen')
    
    ax4.set_title('Sample Frame Movement Constraints')
    ax4.set_xlabel('Field Position (yards)')
    ax4.set_ylabel('Field Width (yards)')
    
    plt.tight_layout()
    plt.savefig(save_path, bbox_inches='tight', dpi=300)
    plt.close()

# Load data for analysis
print(f"Analyzing environmental constraints for {top_team}...")

# Use notable plays from previous analysis
notable_plays_df = pd.read_csv(os.path.join(os.path.dirname(os.getcwd()), 
                                           'results', f'{top_team}_notable_plays.csv'))

# Analyze constraints for each play
all_constraints = []
for _, play in tqdm(notable_plays_df.iterrows(), desc="Analyzing plays"):
    play_tracking = tracking_df[
        (tracking_df['gameId'] == play['gameId']) & 
        (tracking_df['playId'] == play['playId']) &
        (tracking_df['club'] == top_team)
    ]
    
    # Calculate constraints
    constraints = analyze_constraints(play_tracking)
    
    # Create visualization
    save_path = os.path.join(os.path.dirname(os.getcwd()), 'plots',
                            f"{top_team}_play_{play['playId']}_constraints.png")
    plot_constraints(play_tracking, constraints, save_path)
    
    # Aggregate constraints
    constraints_summary = {
        'gameId': play['gameId'],
        'playId': play['playId'],
        'avg_sideline_dist': np.mean(constraints['sideline_proximity']),
        'min_defender_spacing': np.min(constraints['defender_spacing']),
        'avg_movement_angles': np.mean(constraints['movement_angles'])
    }
    all_constraints.append(constraints_summary)

# Create constraints summary DataFrame
constraints_df = pd.DataFrame(all_constraints)

# Save results
constraints_df.to_csv(os.path.join(os.path.dirname(os.getcwd()), 'results',
                                  f'{top_team}_constraints_analysis.csv'), index=False)

print("\nConstraints Analysis Summary:")
print(constraints_df.describe().round(2))
print(f"\nFinal memory usage: {get_memory_usage():.2f} MB")


In [ ]:
# Cell 8: Movement Pattern Classification
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from scipy.signal import savgol_filter
import seaborn as sns
from tqdm import tqdm

def extract_movement_features(play_tracking):
    """Extract key features from defender movements"""
    features = []
    
    # Process each defender's movement
    for player_id, player_data in play_tracking.groupby('nflId'):
        # Sort by frame
        player_data = player_data.sort_values('frameId')
        
        # Calculate movement features
        speed_profile = player_data['s'].values
        direction_profile = player_data['dir'].values
        acceleration_profile = player_data['a'].values
        
        # Smooth profiles using Savitzky-Golay filter
        if len(speed_profile) > 5:  # Need minimum length for filtering
            speed_smooth = savgol_filter(speed_profile, 5, 2)
            acc_smooth = savgol_filter(acceleration_profile, 5, 2)
        else:
            speed_smooth = speed_profile
            acc_smooth = acceleration_profile
            
        # Extract features
        features.append({
            'nflId': player_id,
            'avg_speed': np.mean(speed_smooth),
            'max_speed': np.max(speed_smooth),
            'speed_var': np.var(speed_smooth),
            'avg_acc': np.mean(acc_smooth),
            'max_acc': np.max(acc_smooth),
            'direction_changes': np.sum(np.abs(np.diff(direction_profile)) > 10),
            'total_distance': np.sum(player_data['dis'])
        })
    
    return pd.DataFrame(features)

def classify_movements(features, n_clusters=4):
    """Classify movement patterns using KMeans"""
    # Standardize features
    scaler = StandardScaler()
    X = scaler.fit_transform(features.drop('nflId', axis=1))
    
    # Apply KMeans clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    labels = kmeans.fit_predict(X)
    
    # Add cluster labels to features
    features['pattern'] = labels
    
    return features, kmeans.cluster_centers_

def plot_pattern_analysis(features, centers, save_path):
    """Create visualization of movement patterns"""
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
    
    # 1. Speed vs Acceleration by Pattern
    scatter1 = ax1.scatter(features['avg_speed'], features['avg_acc'], 
                          c=features['pattern'], cmap='viridis')
    ax1.set_xlabel('Average Speed (yards/s)')
    ax1.set_ylabel('Average Acceleration (yards/s²)')
    ax1.set_title('Speed vs Acceleration Patterns')
    plt.colorbar(scatter1, ax=ax1, label='Pattern')

    # 2. Direction Changes Distribution
    sns.boxplot(data=features, x='pattern', y='direction_changes', ax=ax2)
    ax2.set_title('Direction Changes by Pattern')
    ax2.set_xlabel('Pattern')
    ax2.set_ylabel('Number of Direction Changes')

    # 3. Pattern Characteristics Heatmap
    pattern_means = features.groupby('pattern').mean()
    sns.heatmap(pattern_means.drop('nflId', axis=1), 
                annot=True, fmt='.2f', cmap='YlOrRd', ax=ax3)
    ax3.set_title('Pattern Characteristics')

    # 4. Distance vs Speed Variability
    scatter2 = ax4.scatter(features['total_distance'], features['speed_var'],
                          c=features['pattern'], cmap='viridis', alpha=0.6)
    ax4.set_xlabel('Total Distance (yards)')
    ax4.set_ylabel('Speed Variability')
    ax4.set_title('Distance vs Speed Variability')
    plt.colorbar(scatter2, ax=ax4, label='Pattern')

    plt.tight_layout()
    plt.savefig(save_path, bbox_inches='tight', dpi=300)
    plt.close()

# Load data and process patterns
print(f"Analyzing movement patterns for {top_team}...")

# Use notable plays from previous analysis
notable_plays_df = pd.read_csv(os.path.join(os.path.dirname(os.getcwd()),
                                           'results', f'{top_team}_notable_plays.csv'))

# Analyze patterns for each play
all_patterns = []
for _, play in tqdm(notable_plays_df.iterrows(), desc="Analyzing patterns"):
    play_tracking = tracking_df[
        (tracking_df['gameId'] == play['gameId']) & 
        (tracking_df['playId'] == play['playId']) &
        (tracking_df['club'] == top_team)
    ]
    
    # Extract features and classify movements
    features = extract_movement_features(play_tracking)
    features['gameId'] = play['gameId']
    features['playId'] = play['playId']
    
    classified_features, centers = classify_movements(features.drop(['gameId', 'playId'], axis=1))
    classified_features['gameId'] = play['gameId']
    classified_features['playId'] = play['playId']
    
    all_patterns.append(classified_features)
    
    # Create visualization
    save_path = os.path.join(os.path.dirname(os.getcwd()), 'plots',
                            f"{top_team}_play_{play['playId']}_patterns.png")
    plot_pattern_analysis(classified_features, centers, save_path)

# Combine all patterns
patterns_df = pd.concat(all_patterns, ignore_index=True)

# Calculate pattern statistics
pattern_stats = patterns_df.groupby('pattern').agg({
    'avg_speed': ['mean', 'std'],
    'max_speed': ['mean', 'std'],
    'direction_changes': ['mean', 'std'],
    'total_distance': ['mean', 'std']
}).round(2)

# Save results
patterns_df.to_csv(os.path.join(os.path.dirname(os.getcwd()), 'results',
                               f'{top_team}_movement_patterns.csv'), index=False)
pattern_stats.to_csv(os.path.join(os.path.dirname(os.getcwd()), 'results',
                                 f'{top_team}_pattern_statistics.csv'))

print("\nPattern Analysis Summary:")
print("\nPattern Distribution:")
print(patterns_df['pattern'].value_counts().sort_index())
print("\nPattern Statistics:")
print(pattern_stats)
print(f"\nFinal memory usage: {get_memory_usage():.2f} MB")


In [ ]:
# Cell 9: Efficiency Metrics and Optimal Path Analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from scipy.interpolate import interp1d
import seaborn as sns
from tqdm import tqdm

def calculate_optimal_path(start_pos, end_pos, obstacles, n_points=50):
    """Calculate theoretically optimal path avoiding obstacles"""
    # Create smooth path between start and end
    t = np.linspace(0, 1, n_points)
    path_x = np.linspace(start_pos[0], end_pos[0], n_points)
    path_y = np.linspace(start_pos[1], end_pos[1], n_points)
    
    # Adjust path to avoid obstacles
    for i in range(1, n_points-1):
        point = np.array([path_x[i], path_y[i]])
        
        # Check distance to obstacles
        distances = cdist([point], obstacles)
        min_dist = np.min(distances)
        
        if min_dist < 2:  # If too close to obstacle
            # Find nearest obstacle
            nearest_idx = np.argmin(distances)
            obstacle = obstacles[nearest_idx]
            
            # Calculate avoidance vector
            avoid_vector = point - obstacle
            avoid_vector = avoid_vector / np.linalg.norm(avoid_vector)
            
            # Adjust point position
            path_x[i] = point[0] + avoid_vector[0] * (2 - min_dist)
            path_y[i] = point[1] + avoid_vector[1] * (2 - min_dist)
    
    return np.column_stack([path_x, path_y])

def calculate_efficiency_metrics(actual_path, optimal_path):
    """Calculate movement efficiency metrics"""
    # Calculate path lengths
    actual_length = np.sum(np.sqrt(np.sum(np.diff(actual_path, axis=0)**2, axis=1)))
    optimal_length = np.sum(np.sqrt(np.sum(np.diff(optimal_path, axis=0)**2, axis=1)))
    
    # Calculate average deviation from optimal path
    deviations = []
    for actual_point in actual_path:
        distances = cdist([actual_point], optimal_path)
        deviations.append(np.min(distances))
    
    metrics = {
        'path_efficiency': optimal_length / actual_length if actual_length > 0 else 0,
        'avg_deviation': np.mean(deviations),
        'max_deviation': np.max(deviations),
        'total_distance': actual_length
    }
    
    return metrics

def plot_efficiency_analysis(actual_path, optimal_path, metrics, save_path):
    """Create visualization comparing actual and optimal paths"""
    plt.figure(figsize=(15, 10))
    
    # Create subplot grid
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
    
    # 1. Path Comparison
    ax1.plot(actual_path[:, 0], actual_path[:, 1], 'b-', label='Actual Path', alpha=0.7)
    ax1.plot(optimal_path[:, 0], optimal_path[:, 1], 'r--', label='Optimal Path', alpha=0.7)
    ax1.scatter(actual_path[0, 0], actual_path[0, 1], c='g', s=100, label='Start')
    ax1.scatter(actual_path[-1, 0], actual_path[-1, 1], c='r', s=100, label='End')
    
    # Add field boundaries
    ax1.plot([0, 0], [0, 53.3], 'white', alpha=0.5)
    ax1.plot([120, 120], [0, 53.3], 'white', alpha=0.5)
    ax1.plot([0, 120], [0, 0], 'white', alpha=0.5)
    ax1.plot([0, 120], [53.3, 53.3], 'white', alpha=0.5)
    ax1.set_facecolor('forestgreen')
    
    ax1.set_title('Path Comparison')
    ax1.set_xlabel('Field Position (yards)')
    ax1.set_ylabel('Field Position (yards)')
    ax1.legend()
    
    # 2. Deviation Over Time
    deviations = [np.min(cdist([p], optimal_path)) for p in actual_path]
    ax2.plot(deviations, 'b-')
    ax2.set_title('Path Deviation Over Time')
    ax2.set_xlabel('Time Step')
    ax2.set_ylabel('Deviation (yards)')
    
    # 3. Efficiency Metrics
    metrics_list = list(metrics.items())  # Convert dict_items to list
    ax3.axis('off')
    ax3.table(cellText=[[v] for k, v in metrics_list if isinstance(v, (int, float))],
              rowLabels=[k for k, v in metrics_list if isinstance(v, (int, float))],
              colLabels=['Value'],
              loc='center')
    ax3.set_title('Efficiency Metrics')
    
    # 4. Cumulative Distance
    actual_cum_dist = np.cumsum(np.sqrt(np.sum(np.diff(actual_path, axis=0)**2, axis=1)))
    optimal_cum_dist = np.cumsum(np.sqrt(np.sum(np.diff(optimal_path, axis=0)**2, axis=1)))
    
    ax4.plot(actual_cum_dist, 'b-', label='Actual')
    ax4.plot(optimal_cum_dist, 'r--', label='Optimal')
    ax4.set_title('Cumulative Distance')
    ax4.set_xlabel('Time Step')
    ax4.set_ylabel('Distance (yards)')
    ax4.legend()
    
    plt.tight_layout()
    plt.savefig(save_path, bbox_inches='tight', dpi=300)
    plt.close()

# Load data for analysis
print(f"Analyzing movement efficiency for {top_team}...")

# Use notable plays from previous analysis
notable_plays_df = pd.read_csv(os.path.join(os.path.dirname(os.getcwd()),
                                           'results', f'{top_team}_notable_plays.csv'))

# Analyze efficiency for each play
all_efficiency_metrics = []
for _, play in tqdm(notable_plays_df.iterrows(), desc="Analyzing efficiency"):
    play_tracking = tracking_df[
        (tracking_df['gameId'] == play['gameId']) & 
        (tracking_df['playId'] == play['playId']) &
        (tracking_df['club'] == top_team)
    ]
    
    # Process each defender
    for player_id, player_data in play_tracking.groupby('nflId'):
        # Get actual path
        actual_path = player_data[['x', 'y']].values
        
        # Get other players' positions as obstacles
        other_players = play_tracking[play_tracking['nflId'] != player_id]
        obstacles = other_players[['x', 'y']].values
        
        # Calculate optimal path
        optimal_path = calculate_optimal_path(actual_path[0], actual_path[-1], obstacles)
        
        # Calculate metrics
        metrics = calculate_efficiency_metrics(actual_path, optimal_path)
        metrics.update({
            'gameId': play['gameId'],
            'playId': play['playId'],
            'nflId': player_id
        })
        all_efficiency_metrics.append(metrics)
        
        # Create visualization
        save_path = os.path.join(os.path.dirname(os.getcwd()), 'plots',
                                f"{top_team}_play_{play['playId']}_player_{player_id}_efficiency.png")
        plot_efficiency_analysis(actual_path, optimal_path, metrics, save_path)

# Create efficiency DataFrame
efficiency_df = pd.DataFrame(all_efficiency_metrics)

# Calculate summary statistics
efficiency_summary = efficiency_df.groupby(['gameId', 'playId']).agg({
    'path_efficiency': ['mean', 'std'],
    'avg_deviation': ['mean', 'std'],
    'total_distance': ['mean', 'sum']
}).round(3)

# Save results
efficiency_df.to_csv(os.path.join(os.path.dirname(os.getcwd()), 'results',
                                 f'{top_team}_efficiency_metrics.csv'), index=False)
efficiency_summary.to_csv(os.path.join(os.path.dirname(os.getcwd()), 'results',
                                     f'{top_team}_efficiency_summary.csv'))

print("\nEfficiency Analysis Summary:")
print("\nOverall Metrics:")
print(efficiency_df[['path_efficiency', 'avg_deviation', 'total_distance']].describe().round(3))
print(f"\nFinal memory usage: {get_memory_usage():.2f} MB")


In [ ]:
# Cell 10: Animation Generation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.patches import Circle, Rectangle
from tqdm import tqdm

def create_field():
    """Create football field for animation"""
    fig, ax = plt.subplots(figsize=(15, 8))
    
    # Field boundaries
    ax.plot([0, 0], [0, 53.3], 'white', alpha=0.5)
    ax.plot([120, 120], [0, 53.3], 'white', alpha=0.5)
    ax.plot([0, 120], [0, 0], 'white', alpha=0.5)
    ax.plot([0, 120], [53.3, 53.3], 'white', alpha=0.5)
    
    ax.set_facecolor('forestgreen')
    ax.set_xlim(-5, 125)
    ax.set_ylim(-5, 58.3)
    
    return fig, ax

def create_defender_animation(play_tracking, save_path):
    """Create animation of defensive movement"""
    fig, ax = create_field()
    
    # Prepare defender data
    defenders = {}
    trails = {}
    for player_id in play_tracking['nflId'].unique():
        player_data = play_tracking[play_tracking['nflId'] == player_id]
        defenders[player_id] = player_data[['frameId', 'x', 'y']].values
        trails[player_id] = []
    
    # Initialize plots
    defender_plots = {}
    trail_plots = {}
    for player_id in defenders:
        defender_plots[player_id], = ax.plot([], [], 'wo', markersize=10)
        trail_plots[player_id], = ax.plot([], [], 'w-', alpha=0.3)
    
    # Add time display
    time_text = ax.text(5, 55, '', color='white', fontsize=12)
    
    def init():
        for player_id in defenders:
            defender_plots[player_id].set_data([], [])
            trail_plots[player_id].set_data([], [])
        time_text.set_text('')
        return list(defender_plots.values()) + list(trail_plots.values()) + [time_text]
    
    def animate(frame):
        for player_id in defenders:
            if frame < len(defenders[player_id]):
                x, y = defenders[player_id][frame][1:3]
                defender_plots[player_id].set_data([x], [y])
                
                # Update trails
                trails[player_id].append([x, y])
                if len(trails[player_id]) > 10:  # Trail length
                    trails[player_id].pop(0)
                trail_data = np.array(trails[player_id])
                trail_plots[player_id].set_data(trail_data[:, 0], trail_data[:, 1])
        
        time_text.set_text(f'Frame: {frame}')
        return list(defender_plots.values()) + list(trail_plots.values()) + [time_text]
    
    # Create animation
    anim = animation.FuncAnimation(fig, animate, init_func=init,
                                 frames=len(defenders[list(defenders.keys())[0]]),
                                 interval=50, blit=True)
    
    # Save animation
    writer = animation.PillowWriter(fps=20)
    anim.save(save_path, writer=writer)
    plt.close()

def create_zone_coverage_animation(play_tracking, save_path):
    """Create animation of defensive zone coverage"""
    fig, ax = create_field()
    
    frames = play_tracking['frameId'].unique()
    x = np.linspace(0, 120, 50)
    y = np.linspace(0, 53.3, 25)
    X, Y = np.meshgrid(x, y)
    
    # Initialize heatmap
    heatmap = ax.pcolormesh(X, Y, np.zeros_like(X), cmap='YlOrRd', alpha=0.5)
    plt.colorbar(heatmap, label='Coverage Intensity')
    
    def update_heatmap(frame):
        frame_data = play_tracking[play_tracking['frameId'] == frame]
        Z = np.zeros_like(X)
        
        # Calculate coverage intensity
        for _, player in frame_data.iterrows():
            dx = X - player['x']
            dy = Y - player['y']
            Z += np.exp(-(dx**2 + dy**2) / 100)  # Gaussian coverage
        
        heatmap.set_array(Z.ravel())
        return [heatmap]
    
    # Create animation
    anim = animation.FuncAnimation(fig, update_heatmap, frames=frames,
                                 interval=50, blit=True)
    
    # Save animation
    writer = animation.PillowWriter(fps=20)
    anim.save(save_path, writer=writer)
    plt.close()

# Load best play data for animation
print(f"Creating animations for {top_team}...")

# Get the most interesting play based on efficiency metrics
best_play = efficiency_df.nlargest(1, 'path_efficiency')
play_id = best_play['playId'].iloc[0]
game_id = best_play['gameId'].iloc[0]

# Load play tracking data
play_tracking = tracking_df[
    (tracking_df['gameId'] == game_id) & 
    (tracking_df['playId'] == play_id) &
    (tracking_df['club'] == top_team)
].sort_values('frameId')

print(f"\nCreating animations for play {play_id}")

# Create defender movement animation
defender_animation_path = os.path.join(os.path.dirname(os.getcwd()), 
                                     'animations', 
                                     f'{top_team}_play_{play_id}_movement.gif')
create_defender_animation(play_tracking, defender_animation_path)

# Create zone coverage animation
coverage_animation_path = os.path.join(os.path.dirname(os.getcwd()), 
                                     'animations', 
                                     f'{top_team}_play_{play_id}_coverage.gif')
create_zone_coverage_animation(play_tracking, coverage_animation_path)

print("\nAnimation Summary:")
print(f"Defender Movement Animation: {defender_animation_path}")
print(f"Zone Coverage Animation: {coverage_animation_path}")
print(f"\nFinal memory usage: {get_memory_usage():.2f} MB")


In [ ]:
# Cell 11: Annotated Animation Generation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.patches import Circle, Rectangle, FancyArrowPatch
from matplotlib.text import Annotation
from tqdm import tqdm

def create_annotated_animation(play_tracking, play_info, save_path):
    """Create annotated animation of defensive movement"""
    fig, ax = create_field()
    
    # Prepare defender data
    defenders = {}
    trails = {}
    for player_id in play_tracking['nflId'].unique():
        player_data = play_tracking[play_tracking['nflId'] == player_id]
        defenders[player_id] = player_data[['frameId', 'x', 'y', 's', 'dir']].values
        trails[player_id] = []
    
    # Initialize plots
    defender_plots = {}
    trail_plots = {}
    for player_id in defenders:
        defender_plots[player_id], = ax.plot([], [], 'wo', markersize=10)
        trail_plots[player_id], = ax.plot([], [], 'w-', alpha=0.3)
    
    # Add display elements
    time_text = ax.text(5, 55, '', color='white', fontsize=12)
    speed_text = ax.text(80, 55, '', color='white', fontsize=12)
    
    # Add annotation arrows and text boxes
    annotations = []
    def add_annotation(x, y, text, xytext_offset=(30, 30)):
        ann = ax.annotate(
            text,
            xy=(x, y),
            xytext=(x + xytext_offset[0], y + xytext_offset[1]),
            color='white',
            bbox=dict(facecolor='black', alpha=0.7),
            arrowprops=dict(arrowstyle='->'),
            animated=True
        )
        annotations.append(ann)
        return ann
    
    def init():
        for player_id in defenders:
            defender_plots[player_id].set_data([], [])
            trail_plots[player_id].set_data([], [])
        time_text.set_text('')
        speed_text.set_text('')
        for ann in annotations:
            ann.set_visible(False)
        return (list(defender_plots.values()) + 
                list(trail_plots.values()) + 
                [time_text, speed_text] + 
                annotations)
    
    def animate(frame):
        # Update defender positions and trails
        max_speed = 0
        for player_id in defenders:
            if frame < len(defenders[player_id]):
                x, y = defenders[player_id][frame][1:3]
                speed = defenders[player_id][frame][3]
                max_speed = max(max_speed, speed)
                
                defender_plots[player_id].set_data([x], [y])
                
                # Update trails
                trails[player_id].append([x, y])
                if len(trails[player_id]) > 10:
                    trails[player_id].pop(0)
                trail_data = np.array(trails[player_id])
                trail_plots[player_id].set_data(trail_data[:, 0], trail_data[:, 1])
        
        # Update display texts
        time_text.set_text(f'Frame: {frame}')
        speed_text.set_text(f'Max Speed: {max_speed:.1f} yards/s')
        
        # Update annotations based on frame
        for ann in annotations:
            ann.set_visible(True)
            
        # Add frame-specific annotations
        if frame == 0:
            add_annotation(
                defenders[list(defenders.keys())[0]][frame][1],
                defenders[list(defenders.keys())[0]][frame][2],
                'Starting Position'
            )
        elif frame == len(defenders[list(defenders.keys())[0]])//2:
            add_annotation(
                defenders[list(defenders.keys())[0]][frame][1],
                defenders[list(defenders.keys())[0]][frame][2],
                'Coverage Adjustment'
            )
        
        return (list(defender_plots.values()) + 
                list(trail_plots.values()) + 
                [time_text, speed_text] + 
                annotations)
    
    # Create animation
    anim = animation.FuncAnimation(fig, animate, init_func=init,
                                 frames=len(defenders[list(defenders.keys())[0]]),
                                 interval=50, blit=True)
    
    # Add title with play information
    plt.title(f"{top_team} Defensive Movement Analysis\n"
             f"Play ID: {play_info['playId']}, Game ID: {play_info['gameId']}")
    
    # Save animation
    writer = animation.PillowWriter(fps=20)
    anim.save(save_path, writer=writer)
    plt.close()

# Create annotated version of the movement animation
print("Creating annotated animation...")

# Get play information
best_play_info = {
    'playId': play_id,
    'gameId': game_id
}

# Create annotated animation
annotated_animation_path = os.path.join(os.path.dirname(os.getcwd()), 
                                      'animations', 
                                      f'{top_team}_play_{play_id}_annotated.gif')

create_annotated_animation(play_tracking, best_play_info, annotated_animation_path)

print(f"\nAnnotated animation saved to: {annotated_animation_path}")
print(f"Final memory usage: {get_memory_usage():.2f} MB")


In [ ]:
# Cell 12: Animation Documentation and Selection
import pandas as pd
import json
import os
from datetime import datetime
import numpy as np

# Add JSON encoder to handle numpy types
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (np.int64, np.int32, np.int16, np.int8)):
            return int(obj)
        if isinstance(obj, (np.float64, np.float32)):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NumpyEncoder, self).default(obj)

def create_animation_documentation(team, play_id, game_id, animation_type):
    """Create detailed documentation for each selected animation"""
    
    # Load relevant metrics for this play
    efficiency_metrics = pd.read_csv(os.path.join(os.path.dirname(os.getcwd()), 
                                                'results', 
                                                f'{team}_efficiency_metrics.csv'))
    play_metrics = efficiency_metrics[
        (efficiency_metrics['playId'] == play_id) & 
        (efficiency_metrics['gameId'] == game_id)
    ].iloc[0]
    
    # Load play information
    plays_df = pd.read_csv(os.path.join(DATA_DIR, 'plays.csv'))
    play_info = plays_df[
        (plays_df['gameId'] == game_id) & 
        (plays_df['playId'] == play_id)
    ].iloc[0]
    
    documentation = {
        'animation_id': f"{team}_play_{play_id}_{animation_type}",
        'team': team,
        'play_id': int(play_id),  # Convert np.int64 to regular int
        'game_id': int(game_id),  # Convert np.int64 to regular int
        'animation_type': animation_type,
        'metrics': {
            'path_efficiency': float(play_metrics['path_efficiency']),
            'avg_deviation': float(play_metrics['avg_deviation']),
            'total_distance': float(play_metrics['total_distance'])
        },
        'play_context': {
            'quarter': int(play_info['quarter']),
            'down': int(play_info['down']),
            'yards_to_go': int(play_info['yardsToGo']),
            'play_description': str(play_info['playDescription'])
        },
        'key_features': {
            'movement_patterns': [
                "Initial defensive alignment",
                "Coverage adjustment response",
                "Defender spacing maintenance",
                "Route recognition reaction",
                "Recovery movement patterns"
            ],
            'technical_highlights': [
                "Speed variations during coverage",
                "Directional changes",
                "Zone handoff execution",
                "Space maintenance discipline"
            ]
        },
        'markdown_description': f"""
## Defensive Movement Analysis: {team} Play {play_id}

### Play Context
- Quarter: {int(play_info['quarter'])}
- Down: {int(play_info['down'])}
- Yards to Go: {int(play_info['yardsToGo'])}
- Situation: {str(play_info['playDescription'])}

### Key Defensive Features
1. **Initial Formation**
   - Defenders aligned in {play_info['defensiveTeam']} base defense
   - Structured spacing creating optimal coverage zones

2. **Movement Efficiency**
   - Path Efficiency: {float(play_metrics['path_efficiency']):.2f}
   - Average Deviation: {float(play_metrics['avg_deviation']):.2f} yards
   - Total Distance: {float(play_metrics['total_distance']):.2f} yards

3. **Coverage Execution**
   - Synchronized defensive adjustments
   - Maintained zone integrity throughout the play
   - Effective space control and gap discipline

### Technical Analysis
- White dots represent defender positions
- Trailing lines show recent movement paths
- Speed indicators display instantaneous velocity
- Coverage zones illustrated by position relationships
"""
    }
    
    return documentation

# Document our selected animations
selected_animations = [
    {
        'team': top_team,
        'play_id': play_id,
        'game_id': game_id,
        'animation_type': 'movement'
    }
    # Add more animations here if we select others
]

# Create documentation for each selected animation
animation_docs = []
for anim in selected_animations:
    doc = create_animation_documentation(
        anim['team'],
        anim['play_id'],
        anim['game_id'],
        anim['animation_type']
    )
    animation_docs.append(doc)

# Save documentation using custom encoder
docs_path = os.path.join(os.path.dirname(os.getcwd()), 
                        'results', 
                        'selected_animations_documentation.json')
with open(docs_path, 'w') as f:
    json.dump(animation_docs, f, indent=4, cls=NumpyEncoder)

# Print markdown for each selected animation
print("\nAnimation Documentation for Kaggle Notebook:")
print("\nNote: These descriptions will be used in the final markdown cells")
print("Each animation counts toward our 9-visualization limit\n")

for doc in animation_docs:
    print(doc['markdown_description'])
    print("\n" + "="*80 + "\n")

print(f"\nDocumentation saved to: {docs_path}")
print(f"Final memory usage: {get_memory_usage():.2f} MB")

# Create a visualization count tracker
viz_count = {
    'plots': len(os.listdir(os.path.join(os.path.dirname(os.getcwd()), 'plots'))),
    'animations': len(selected_animations),
    'total': 0
}
viz_count['total'] = viz_count['plots'] + viz_count['animations']

print("\nVisualization Count:")
print(f"Plots: {viz_count['plots']}")
print(f"Animations: {viz_count['animations']}")
print(f"Total: {viz_count['total']}/9 allowed visualizations")


In [ ]:
# Cell 13: Final Visualization Selection and Organization
import pandas as pd
import numpy as np
import os
import shutil
from datetime import datetime

def analyze_visualization_impact(viz_type, filename, metrics_df):
    """Analyze the significance of each visualization"""
    if viz_type == 'plot':
        # Extract play_id from filename if present
        play_id = filename.split('_play_')[-1].split('_')[0] if '_play_' in filename else None
        
        if play_id and play_id.isdigit():
            # Get metrics for this play
            play_metrics = metrics_df[metrics_df['playId'] == int(play_id)]
            if not play_metrics.empty:
                return {
                    'filename': filename,
                    'type': viz_type,
                    'impact_score': float(play_metrics['path_efficiency'].iloc[0]),
                    'context': 'Play-specific visualization'
                }
        
        # For non-play-specific plots
        return {
            'filename': filename,
            'type': viz_type,
            'impact_score': 1.0 if 'summary' in filename else 0.5,
            'context': 'Summary visualization' if 'summary' in filename else 'Supporting visualization'
        }
    
    else:  # animation
        return {
            'filename': filename,
            'type': 'animation',
            'impact_score': 1.0,  # Animations are typically high-impact
            'context': 'Key defensive movement visualization'
        }

def select_final_visualizations(top_team, max_viz=9):
    """Select the most impactful visualizations within the limit"""
    # Load metrics for reference
    efficiency_df = pd.read_csv(os.path.join(os.path.dirname(os.getcwd()), 
                                           'results', 
                                           f'{top_team}_efficiency_metrics.csv'))
    
    # Get all visualizations
    plots_dir = os.path.join(os.path.dirname(os.getcwd()), 'plots')
    animations_dir = os.path.join(os.path.dirname(os.getcwd()), 'animations')
    
    all_visualizations = []
    
    # Analyze plots
    for filename in os.listdir(plots_dir):
        if filename.endswith(('.png', '.jpg')):
            analysis = analyze_visualization_impact('plot', filename, efficiency_df)
            all_visualizations.append(analysis)
    
    # Analyze animations
    for filename in os.listdir(animations_dir):
        if filename.endswith('.gif'):
            analysis = analyze_visualization_impact('animation', filename, efficiency_df)
            all_visualizations.append(analysis)
    
    # Sort by impact score
    sorted_viz = sorted(all_visualizations, 
                       key=lambda x: x['impact_score'], 
                       reverse=True)
    
    # Select top visualizations while ensuring we have key categories
    final_selection = []
    categories_needed = {
        'summary': 1,
        'animation': 1,
        'play_specific': 3
    }
    
    for viz in sorted_viz:
        if len(final_selection) >= max_viz:
            break
            
        # Ensure we have necessary category representation
        if 'summary' in viz['context'].lower() and categories_needed['summary'] > 0:
            final_selection.append(viz)
            categories_needed['summary'] -= 1
        elif viz['type'] == 'animation' and categories_needed['animation'] > 0:
            final_selection.append(viz)
            categories_needed['animation'] -= 1
        elif 'play-specific' in viz['context'].lower() and categories_needed['play_specific'] > 0:
            final_selection.append(viz)
            categories_needed['play_specific'] -= 1
        elif sum(categories_needed.values()) < max_viz - len(final_selection):
            final_selection.append(viz)
    
    return final_selection

def create_visualization_manifest(selected_viz):
    """Create a markdown-ready manifest of selected visualizations"""
    manifest = "# Selected Visualizations\n\n"
    
    # Group by type
    plots = [v for v in selected_viz if v['type'] == 'plot']
    animations = [v for v in selected_viz if v['type'] == 'animation']
    
    # Add plots section
    manifest += "## Static Visualizations\n\n"
    for i, plot in enumerate(plots, 1):
        manifest += f"{i}. **{plot['filename']}**\n"
        manifest += f"   - Context: {plot['context']}\n"
        manifest += f"   - Impact Score: {plot['impact_score']:.2f}\n\n"
    
    # Add animations section
    manifest += "## Animations\n\n"
    for i, anim in enumerate(animations, 1):
        manifest += f"{i}. **{anim['filename']}**\n"
        manifest += f"   - Context: {anim['context']}\n"
        manifest += f"   - Impact Score: {anim['impact_score']:.2f}\n\n"
    
    return manifest

# Execute visualization selection
print("Selecting final visualizations...")
final_visualizations = select_final_visualizations(top_team)

# Create and save manifest
manifest = create_visualization_manifest(final_visualizations)
manifest_path = os.path.join(os.path.dirname(os.getcwd()), 
                            'results', 
                            'final_visualization_manifest.md')
with open(manifest_path, 'w') as f:
    f.write(manifest)

# Create directory for final selections
final_dir = os.path.join(os.path.dirname(os.getcwd()), 'final_visualizations')
os.makedirs(final_dir, exist_ok=True)

# Copy selected visualizations to final directory
for viz in final_visualizations:
    source_dir = os.path.join(os.path.dirname(os.getcwd()),
                             'plots' if viz['type'] == 'plot' else 'animations')
    source_path = os.path.join(source_dir, viz['filename'])
    dest_path = os.path.join(final_dir, viz['filename'])
    shutil.copy2(source_path, dest_path)

print("\nFinal Visualization Selection:")
print(f"Total selected: {len(final_visualizations)}/9")
print("\nVisualization manifest saved to:", manifest_path)
print("Selected visualizations copied to:", final_dir)
print("\nManifest Preview:")
print("="*80)
print(manifest[:500] + "...\n")
print("="*80)
print(f"\nFinal memory usage: {get_memory_usage():.2f} MB")


In [ ]:
# Cell 14 (Revised): Kaggle Notebook Preparation with Mathematical Foundation
import pandas as pd
import numpy as np
import json
import os
from datetime import datetime

def create_notebook_sections():
    """Create structured markdown content for Kaggle notebook with mathematical detail"""
    
    # Load previous content
    with open(os.path.join(os.path.dirname(os.getcwd()), 
                          'results', 
                          'final_visualization_manifest.md'), 'r') as f:
        viz_manifest = f.read()
    
    with open(os.path.join(os.path.dirname(os.getcwd()),
                          'results',
                          'selected_animations_documentation.json'), 'r') as f:
        animation_docs = json.load(f)

    notebook_content = {
        'introduction': f"""# Ecological Movement Modeling in NFL Defense: A Mathematical Analysis
        
## Introduction
This analysis explores defensive movement patterns in the NFL through the lens of ecological 
movement modeling, focusing on the {top_team}'s defensive unit. By applying principles from 
ecological mathematics and movement theory, we uncover the underlying patterns that govern 
effective defensive coverage.

The movement of defensive players can be understood as a complex dynamical system, where 
individual agents (defenders) respond to both local and global stimuli while maintaining 
coordinated coverage of spatial zones.
""",
        
        'mathematical_foundation': """## Mathematical Framework

### 1. Movement Dynamics
Individual defender movement can be modeled using a stochastic differential equation:

$$ \\frac{dx}{dt} = v(t) + \\sigma(x,t)\\xi(t) $$

Where:
- $x(t)$ represents position vector
- $v(t)$ is the velocity field
- $\\sigma(x,t)$ captures environmental influence
- $\\xi(t)$ represents random fluctuations

### 2. Coverage Zone Optimization
The optimal coverage configuration minimizes the exposure function:

$$ E(\\mathbf{X}) = \\int_\\Omega \\min_{i} \\|x - x_i\\| dx $$

Where:
- $\\mathbf{X} = (x_1,...,x_n)$ represents defender positions
- $\\Omega$ is the field area
- $\\|x - x_i\\|$ is the distance to defender $i$

### 3. Collective Motion Metrics
Team coordination is quantified through the order parameter:

$$ \\psi(t) = \\frac{1}{N}\\left|\\sum_{i=1}^N e^{i\\theta_i(t)}\\right| $$

Where:
- $N$ is the number of defenders
- $\\theta_i(t)$ is the movement direction of defender $i$

### 4. Path Efficiency Analysis
Movement efficiency is calculated using the ratio:

$$ \\eta = \\frac{\\|x_f - x_0\\|}{\\int_0^T \\|v(t)\\| dt} $$

Where:
- $x_0$ and $x_f$ are initial and final positions
- $T$ is the play duration
- $\\|v(t)\\|$ is instantaneous speed

### 5. Zone Coverage Model
Coverage quality is evaluated using a potential field approach:

$$ \\phi(x) = \\sum_{i=1}^N A_i e^{-\\|x-x_i\\|^2/2\\sigma_i^2} $$

Where:
- $A_i$ is defender effectiveness
- $\\sigma_i$ is coverage radius
""",

        'methodology': f"""## Methodology: From Theory to Practice

### Implementation of Mathematical Models
Our analysis translates these mathematical principles into practical metrics:

1. **Movement Pattern Analysis**
   - Discretized version of continuous movement equations
   - Numerical integration of path efficiencies
   - Statistical analysis of coverage patterns

2. **Spatial Analysis Framework**
   - Voronoi tessellation of defensive coverage
   - Dynamic time warping for pattern matching
   - Kernel density estimation for zone coverage

3. **Efficiency Calculations**
   - Path integral computation
   - Coverage optimization algorithms
   - Coordination metric evaluation

### Data Processing Pipeline
1. Position tracking at 10 Hz
2. Velocity and acceleration computation
3. Coverage zone calculation
4. Pattern recognition and classification
""",
        
        'movement_analysis': """## Defensive Movement Analysis

### Individual Defender Dynamics
We observe that defender movements follow a modified Ornstein-Uhlenbeck process:

$$ dv = -\\gamma(v - v_p)dt + \\sigma dW_t $$

Where:
- $v_p$ is preferred velocity
- $\\gamma$ is response rate
- $W_t$ is a Wiener process

This mathematical framework reveals how defenders balance between:
1. Maintaining assigned coverage responsibilities
2. Responding to offensive movements
3. Coordinating with teammates

[Insert Movement Pattern Visualization 1]

The animation above demonstrates these principles in action, showing how theoretical
predictions align with actual defensive movements.
""",

        'coverage_analysis': """## Coverage Zone Analysis

### Optimal Coverage Theory
The defensive unit seeks to minimize the coverage gap function:

$$ G(\\mathbf{X}) = \\max_{y \\in \\Omega} \\min_{i} \\|y - x_i\\| $$

This optimization problem leads to emergent behavior that we observe in the tracking data:

[Insert Coverage Zone Visualization 2]

The heat map above shows the realized coverage density compared to theoretical optimal
distribution.
""",

        'efficiency_metrics': """## Efficiency and Performance Metrics

### Quantitative Performance Measures
Movement efficiency is evaluated through multiple metrics:

1. **Path Efficiency Ratio**:
   $$ \\eta_{path} = \\frac{d_{optimal}}{d_{actual}} $$

2. **Coverage Quality Index**:
   $$ Q_c = \\int_\\Omega \\phi(x)dx $$

3. **Coordination Coefficient**:
   $$ C = \\frac{1}{T}\\int_0^T \\psi(t)dt $$

[Insert Efficiency Comparison Visualization 3]
""",

        'results_interpretation': """## Results and Interpretation

Our mathematical analysis reveals several key insights:

1. **Optimality Conditions**
   The observed defensive movements satisfy the Euler-Lagrange equation:
   $$ \\frac{d}{dt}\\frac{\\partial L}{\\partial \\dot{x}} - \\frac{\\partial L}{\\partial x} = 0 $$
   where $L$ is the Lagrangian of the system.

2. **Pattern Emergence**
   Defensive coordination emerges from local rules following:
   $$ \\dot{x}_i = \\sum_{j \\neq i} f(\\|x_j - x_i\\|)(x_j - x_i) $$

3. **Efficiency Boundaries**
   Performance limits are governed by:
   $$ \\eta_{max} = \\sup_{\\mathbf{X}} \\{\\eta(\\mathbf{X}) : C(\\mathbf{X}) \\geq c_{min}\\} $$

[Insert Pattern Analysis Visualization 4]
""",

        'conclusions': """## Conclusions and Future Directions

Our mathematical analysis demonstrates that NFL defensive movements follow predictable
patterns governed by fundamental principles of collective motion and optimal control theory.
Key findings include:

1. Emergence of optimal coverage patterns through local interactions
2. Quantifiable trade-offs between individual and collective efficiency
3. Predictable response patterns based on environmental constraints

Future work will explore:
- Higher-order movement correlations
- Non-linear response functions
- Stochastic game theory applications
""",

        'github_reference': """## Extended Mathematical Analysis

Complete mathematical derivations, numerical methods, and additional analyses are available
in our GitHub repository. This includes:

1. Detailed mathematical proofs
2. Numerical simulation code
3. Statistical validation tests
4. Extended data visualizations

[GitHub Repository Link to be added]
"""
    }
    
    return notebook_content

# Generate notebook content
print("Generating enhanced Kaggle notebook content...")
notebook_sections = create_notebook_sections()

# Save sections to markdown files
output_dir = os.path.join(os.path.dirname(os.getcwd()), 'kaggle_notebook')
os.makedirs(output_dir, exist_ok=True)

for section_name, content in notebook_sections.items():
    file_path = os.path.join(output_dir, f'{section_name}.md')
    with open(file_path, 'w') as f:
        f.write(content)

# Create word count summary
word_counts = {
    section: len(content.split()) 
    for section, content in notebook_sections.items()
}
total_words = sum(word_counts.values())

print("\nNotebook Section Word Counts:")
for section, count in word_counts.items():
    print(f"{section}: {count} words")
print(f"\nTotal Words: {total_words}/2000")

print("\nNote: LaTeX equations not counted in word count")
print(f"\nNotebook sections saved to: {output_dir}")
print(f"\nFinal memory usage: {get_memory_usage():.2f} MB")


In [ ]:
# Cell 15: Visualization Review and Selection
import os
import pandas as pd
from PIL import Image
import json

def review_visualizations():
    """Review and catalog all generated visualizations"""
    
    plots_dir = os.path.join(os.path.dirname(os.getcwd()), 'plots')
    animations_dir = os.path.join(os.path.dirname(os.getcwd()), 'animations')
    final_dir = os.path.join(os.path.dirname(os.getcwd()), 'final_visualizations')

    # Create catalog of all visualizations
    catalog = {
        'plots': [],
        'animations': [],
        'selected': []
    }

    # Review plots
    print("\nAvailable Plots:")
    print("-" * 80)
    for filename in sorted(os.listdir(plots_dir)):
        if filename.endswith(('.png', '.jpg')):
            file_path = os.path.join(plots_dir, filename)
            img = Image.open(file_path)
            size_mb = os.path.getsize(file_path) / (1024 * 1024)
            
            catalog['plots'].append({
                'filename': filename,
                'dimensions': img.size,
                'size_mb': size_mb,
                'type': 'plot'
            })
            
            print(f"Plot: {filename}")
            print(f"Dimensions: {img.size}")
            print(f"Size: {size_mb:.2f} MB")
            print("-" * 80)

    # Review animations
    print("\nAvailable Animations:")
    print("-" * 80)
    for filename in sorted(os.listdir(animations_dir)):
        if filename.endswith('.gif'):
            file_path = os.path.join(animations_dir, filename)
            size_mb = os.path.getsize(file_path) / (1024 * 1024)
            
            catalog['animations'].append({
                'filename': filename,
                'size_mb': size_mb,
                'type': 'animation'
            })
            
            print(f"Animation: {filename}")
            print(f"Size: {size_mb:.2f} MB")
            print("-" * 80)

    # Review current selections
    if os.path.exists(final_dir):
        print("\nCurrently Selected Visualizations:")
        print("-" * 80)
        for filename in sorted(os.listdir(final_dir)):
            file_path = os.path.join(final_dir, filename)
            size_mb = os.path.getsize(file_path) / (1024 * 1024)
            
            catalog['selected'].append({
                'filename': filename,
                'size_mb': size_mb
            })
            
            print(f"Selected: {filename}")
            print(f"Size: {size_mb:.2f} MB")
            print("-" * 80)

    return catalog

def categorize_visualizations(catalog):
    """Categorize visualizations by their content type"""
    categories = {
        'summary_plots': [],
        'movement_analysis': [],
        'coverage_analysis': [],
        'efficiency_analysis': [],
        'animations': []
    }
    
    for item in catalog['plots']:
        filename = item['filename']
        if 'summary' in filename:
            categories['summary_plots'].append(item)
        elif 'movement' in filename or 'pattern' in filename:
            categories['movement_analysis'].append(item)
        elif 'coverage' in filename:
            categories['coverage_analysis'].append(item)
        elif 'efficiency' in filename:
            categories['efficiency_analysis'].append(item)
    
    categories['animations'] = catalog['animations']
    
    return categories

# Execute review
print("Reviewing all visualizations...")
catalog = review_visualizations()
categories = categorize_visualizations(catalog)

# Print summary by category
print("\nVisualization Summary by Category:")
print("=" * 80)
for category, items in categories.items():
    print(f"\n{category.replace('_', ' ').title()}:")
    for item in items:
        print(f"- {item['filename']} ({item['size_mb']:.2f} MB)")

# Check Kaggle constraints
total_selected = len(catalog['selected'])
print("\nKaggle Constraints Check:")
print(f"Current selections: {total_selected}/9 allowed visualizations")

# Save catalog for reference
catalog_path = os.path.join(os.path.dirname(os.getcwd()), 
                           'results', 
                           'visualization_catalog.json')
with open(catalog_path, 'w') as f:
    json.dump(catalog, f, indent=4)

print(f"\nVisualization catalog saved to: {catalog_path}")
print(f"Final memory usage: {get_memory_usage():.2f} MB")

# Provide recommendations
print("\nRecommended Visualization Selection:")
print("=" * 80)
print("For a coherent narrative, we should include:")
print("1. One overall summary visualization")
print("2. One key defensive movement animation")
print("3. One coverage pattern analysis")
print("4. One efficiency comparison")
print("5. One team-specific analysis")
print("\nWould you like to review any specific category in detail?")


In [ ]:
# Cell 16: Visualization Cleanup and Documentation Update
import os
import shutil
import json

def cleanup_visualizations():
    """Remove unnecessary visualizations and update documentation"""
    final_dir = os.path.join(os.path.dirname(os.getcwd()), 'final_visualizations')
    
    # Files to remove
    remove_files = [
        'SF_play_467_movement.gif',
        'SF_play_1992_formation.png'
    ]
    
    # Remove files
    removed_count = 0
    for file in remove_files:
        file_path = os.path.join(final_dir, file)
        if os.path.exists(file_path):
            os.remove(file_path)
            removed_count += 1
            print(f"Removed: {file}")
    
    # Count remaining visualizations
    remaining_files = os.listdir(final_dir)
    print(f"\nRemaining visualizations: {len(remaining_files)}/9")
    
    return remaining_files

def update_documentation():
    """Update all documentation files to reflect changes"""
    # Update visualization manifest
    manifest_path = os.path.join(os.path.dirname(os.getcwd()), 
                                'results', 
                                'final_visualization_manifest.md')
    
    # Load current manifest
    with open(manifest_path, 'r') as f:
        manifest_content = f.read()
    
    # Remove references to deleted files
    updated_manifest = []
    for line in manifest_content.split('\n'):
        if not any(removed in line for removed in ['467_movement.gif', '1992_formation.png']):
            updated_manifest.append(line)
    
    # Save updated manifest
    with open(manifest_path, 'w') as f:
        f.write('\n'.join(updated_manifest))
    
    # Update Kaggle notebook sections
    notebook_dir = os.path.join(os.path.dirname(os.getcwd()), 'kaggle_notebook')
    for section_file in os.listdir(notebook_dir):
        if section_file.endswith('.md'):
            file_path = os.path.join(notebook_dir, section_file)
            with open(file_path, 'r') as f:
                content = f.read()
            
            # Remove references to deleted files
            updated_content = []
            for line in content.split('\n'):
                if not any(removed in line for removed in ['467_movement.gif', '1992_formation.png']):
                    updated_content.append(line)
            
            with open(file_path, 'w') as f:
                f.write('\n'.join(updated_content))

    print("Documentation updated successfully")

def print_visualization_status():
    """Print current status of visualizations"""
    final_dir = os.path.join(os.path.dirname(os.getcwd()), 'final_visualizations')
    files = sorted(os.listdir(final_dir))
    
    print("\nCurrent Visualizations:")
    print("=" * 80)
    for i, file in enumerate(files, 1):
        file_path = os.path.join(final_dir, file)
        size_mb = os.path.getsize(file_path) / (1024 * 1024)
        print(f"{i}. {file} ({size_mb:.2f} MB)")

# Execute cleanup
print("Cleaning up visualizations...")
remaining_files = cleanup_visualizations()

# Update documentation
print("\nUpdating documentation...")
update_documentation()

# Show current status
print_visualization_status()

# Calculate remaining slots
remaining_slots = 9 - len(remaining_files)
print(f"\nRemaining visualization slots: {remaining_slots}")

if remaining_slots > 0:
    print("\nRecommendations for remaining slots:")
    print("1. Consider adding another coverage analysis visualization")
    print("2. Could include an additional efficiency metric visualization")
    print("3. Might add a comparative analysis visualization")
    print("\nWould you like to review candidates for the remaining slots?")

print(f"\nFinal memory usage: {get_memory_usage():.2f} MB")


In [ ]:
# Cell 17: Create Comparative Analysis Visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.gridspec import GridSpec
from scipy import stats

def create_comparative_analysis():
    """Create comprehensive comparative analysis of defensive performance"""
    # Load efficiency metrics for all teams
    metrics_by_team = {}
    for team in NFC_WEST:
        try:
            file_path = os.path.join(os.path.dirname(os.getcwd()), 
                                   'results', 
                                   f'{team}_efficiency_metrics.csv')
            if os.path.exists(file_path):
                metrics_by_team[team] = pd.read_csv(file_path)
        except:
            print(f"No metrics file found for {team}")
    
    # Create comparison figure
    plt.style.use('dark_background')
    fig = plt.figure(figsize=(15, 10))
    gs = GridSpec(2, 2, figure=fig)
    
    # 1. Efficiency Comparison (top left)
    ax1 = fig.add_subplot(gs[0, 0])
    efficiency_data = {team: metrics['path_efficiency'].mean() 
                      for team, metrics in metrics_by_team.items()}
    efficiency_std = {team: metrics['path_efficiency'].std() 
                     for team, metrics in metrics_by_team.items()}
    
    teams = list(efficiency_data.keys())
    values = list(efficiency_data.values())
    std_values = list(efficiency_std.values())
    
    bars = ax1.bar(teams, values, yerr=std_values, 
                   alpha=0.7, capsize=5)
    bars[teams.index(top_team)].set_color('gold')  # Highlight top team
    ax1.set_title('Path Efficiency by Team')
    ax1.set_ylabel('Average Path Efficiency')
    
    # 2. Coverage Area Comparison (top right)
    ax2 = fig.add_subplot(gs[0, 1])
    coverage_data = {
        team: np.sqrt(metrics['total_distance'].mean()) # Using sqrt for better visualization
        for team, metrics in metrics_by_team.items()
    }
    
    teams = list(coverage_data.keys())
    values = list(coverage_data.values())
    
    bars = ax2.bar(teams, values, alpha=0.7)
    bars[teams.index(top_team)].set_color('gold')
    ax2.set_title('Coverage Area by Team')
    ax2.set_ylabel('Sqrt of Average Coverage (yards)')
    
    # 3. Movement Pattern Distribution (bottom left)
    ax3 = fig.add_subplot(gs[1, 0])
    pattern_data = []
    for team in metrics_by_team:
        metrics = metrics_by_team[team]
        kernel = stats.gaussian_kde(metrics['avg_deviation'])
        x_range = np.linspace(metrics['avg_deviation'].min(), 
                            metrics['avg_deviation'].max(), 100)
        pattern_data.append((x_range, kernel(x_range)))
    
    for i, team in enumerate(metrics_by_team):
        x, y = pattern_data[i]
        ax3.plot(x, y, label=team, alpha=0.7)
    ax3.set_title('Movement Pattern Distribution')
    ax3.set_xlabel('Average Deviation (yards)')
    ax3.set_ylabel('Density')
    ax3.legend()
    
    # 4. Radar Chart of Key Metrics (bottom right)
    ax4 = fig.add_subplot(gs[1, 1], projection='polar')
    metrics_for_radar = ['path_efficiency', 'avg_deviation', 'total_distance']
    
    # Normalize metrics for radar chart
    radar_data = {}
    for team in metrics_by_team:
        metrics = metrics_by_team[team]
        radar_data[team] = [
            metrics['path_efficiency'].mean() / max(m['path_efficiency'].mean() 
                for m in metrics_by_team.values()),
            1 - (metrics['avg_deviation'].mean() / max(m['avg_deviation'].mean() 
                for m in metrics_by_team.values())),  # Invert for better visualization
            metrics['total_distance'].mean() / max(m['total_distance'].mean() 
                for m in metrics_by_team.values())
        ]
    
    angles = np.linspace(0, 2*np.pi, len(metrics_for_radar), endpoint=False)
    
    for team in radar_data:
        values = radar_data[team]
        values += values[:1]  # Complete the polygon
        angles_plot = np.concatenate((angles, [angles[0]]))  # Complete the polygon
        
        ax4.plot(angles_plot, values, label=team, alpha=0.7)
        ax4.fill(angles_plot, values, alpha=0.1)
    
    ax4.set_xticks(angles)
    ax4.set_xticklabels(['Efficiency', 'Precision', 'Coverage'])
    ax4.set_title('Defensive Performance Metrics')
    ax4.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))
    
    # Overall title and adjustments
    plt.suptitle('NFC West Defensive Movement Analysis Comparison', 
                size=16, y=1.02)
    plt.tight_layout()
    
    # Save visualization
    save_path = os.path.join(os.path.dirname(os.getcwd()), 
                            'final_visualizations', 
                            'nfc_west_comparison.png')
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    return save_path

# Create the comparison visualization
print("Creating comparative analysis visualization...")
comparison_path = create_comparative_analysis()

# Update documentation
print("\nUpdating documentation...")
manifest_path = os.path.join(os.path.dirname(os.getcwd()), 
                            'results', 
                            'final_visualization_manifest.md')

with open(manifest_path, 'r') as f:
    manifest_content = f.read()

# Add new visualization to manifest
new_entry = f"""
### Comparative Analysis
- **nfc_west_comparison.png**
  - Comprehensive comparison of defensive movements across NFC West
  - Highlights relative performance in efficiency, coverage, and pattern distribution
  - Demonstrates {top_team}'s defensive movement characteristics in context
"""

with open(manifest_path, 'w') as f:
    f.write(manifest_content + new_entry)

print(f"\nComparative visualization saved to: {comparison_path}")
print("Manifest updated with new visualization")

# Show updated visualization count
final_dir = os.path.join(os.path.dirname(os.getcwd()), 'final_visualizations')
final_count = len(os.listdir(final_dir))
print(f"\nTotal visualizations: {final_count}/9")
print(f"Final memory usage: {get_memory_usage():.2f} MB")
